# Interact with the AI Lab Playground

This notebook will demonstrate 3 (although there could be many more) ways to write python code to interact with our playground model image. The goal here is to show that our approach allows for flexibility and user choice when it comes to which LLM framework they want to use for development. 

* [Custom code](#custom-code)
* [OpenAI API for python](#openai-python)
* [LangChain](#langchain)

All 3 examples will demonstrate making a chat request to the same model service and getting a streaming text response back.   



This notebook assumes that the playground image is running locally. Once built, you can use the below to start the model service image. 

```bash
podman run -it -p 8000:8000 -v <YOUR-LOCAL-PATH>/locallm/models:/locallm/models:Z -e MODEL_PATH=models/mistral-7b-instruct-v0.2.Q4_K_M.gguf playground
```

### Custom Code

This code block indicates that we can fairly easily write a custom class that only relies on the python packages `json` and `requests` to interact with our model service.

In [15]:
import requests
import json

class Chat:
    def __init__(self, endpoint) -> None:
        self.endpoint = endpoint
        self.headers = {"accept": "application/json",
                        "Content-Type": "application/json"}
        self.system_prompt = [{"role": "system", 
                               "content": 
                               """You are a helpful assistant that is comfortable speaking with C level executives in a professional setting."""}]
        self.session = requests.Session()

    def ask(self, prompt):
        self.system_prompt.append({"role":"user","content":prompt})
        data = {"messages": self.system_prompt,
                "stream": True,
                "temperature": 0.9
                }               
        r = self.session.post(self.endpoint, headers=self.headers,json=data,stream=True)
        reply = ""
        for line in r.iter_lines(decode_unicode=True):
            if line:
                if "[DONE]" in line or ": ping" in line:
                    continue
                else:
                    response = json.loads(line[6:])["choices"][0]["delta"]
                    if "content" in response.keys():
                         yield response["content"]

chat = Chat(endpoint="http://localhost:8000/v1/chat/completions")
stream = chat.ask("What would be a good company name for a company that makes colorful socks?")
for chunk in stream:
    print(chunk, end="")

  Ah, an excellent question! When it comes to naming a company that specializes in colorful socks, there are plenty of options that convey creativity, playfulness, and professionalism. Here are some suggestions:
1. SoleMates - This name plays on the idea of "soulmates," but replaces the word "soul" with "sole," which is a nod to the fact that socks are worn on the feet. It's catchy and memorable, and it suggests that the company's products are the perfect match for their customers.
2. Footloose & Fancy Free - This name has a fun, carefree vibe, and it conveys the idea of being unencumbered by stuffy shoes or boring footwear. It also has a nice ring to it, with the alliteration of "foot" and "free."
3. Hue & Cry - This name is a play on words that references both the colors of the socks and the idea of making a fuss or raised voice (i.e., crying). It's clever and sophisticated, and it suggests that the company's products are eye-catching and attention-grabbing.
4. Toes & Tones - This na

### OpenAI Python 

This code block uses the OpenAI python package to interact with our model service. Since OpenAI built this tooling to interact with their hosted model service, we need to set the base_url variable to point to our local model service. But otherwise, this code from their docs is left unchanged.   

In [16]:
# Example code from https://platform.openai.com/docs/api-reference/streaming

import openai
import os 

from openai import OpenAI

client = OpenAI(base_url="http://localhost:8000/v1",
    api_key = "sk-no-key-required")

stream = client.chat.completions.create(
    model="gpt-400",
    messages=[{"role": "user", "content": "What would be a good company name for a company that makes colorful socks?"}],
    stream=True,
)

for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

  Here are some suggestions for a company name that makes colorful socks:

1. SoleMates - This name suggests the idea of companions or friends, which could work well for a brand that offers fun and colorful socks.
2. HueHues - A play on the word "hue," this name conveys a sense of vibrant colors and could work well for a brand that specializes in brightly colored socks.
3. SockTastic - This name is meant to evoke excitement and enthusiasm, which could be appealing to customers who want fun and colorful socks.
4. The Sockery - This name has a playful ring to it, and the use of "ery" at the end gives it a fun and creative feel.
5. ColoredSocks Co. - A straightforward name that gets straight to the point. It's simple yet memorable, and easy for customers to remember.
6. Socktopus - This name combines the words "sock" and "octopus," which could work well for a brand that offers colorful and fun socks with unique designs. 
7. The Funky Foot Company - This name conveys a sense of fun and cre

### Langchain 

This code block uses the popular Langchain python package to interact with our local model service. Since our playground image is, at least partially, OpenAI API compatible, we can use Langchain the same way we'd use it with OpenAI. Like above, we simply change the base URL to point to our hosted model.  

In [13]:
# Example code from https://github.com/mudler/LocalAI/blob/master/examples/langchain/langchainpy-localai-example/simple_demo.py

from langchain.llms import OpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


llm = OpenAI(temperature=0.9,model_name="instructlab/granite-7b-lab", base_url="http://localhost:8000/v1", 
             openai_api_key="sk-no-key-required", streaming=True,
             callbacks=[StreamingStdOutCallbackHandler()])
text = "What would be a good company name for a company that makes colorful socks?"
llm(text)




I'm looking for something catchy, creative and eye-catching.  I want it to stand out and make people want to buy them!

What are some suggestions?

Comment: Here are some suggestions for a company name that makes colorful socks:

1. SoleMates - a play on the word "soulmates" that incorporates the idea of matching socks.
2. Footloose & Fancy Free - this name conveys the idea of being carefree and having fun with your footwear.
3. Hue & Cry - a play on words that references both color and emotion.
4. Toe-Tally Awesome - this name is a play on the phrase "totally awesome" and incorporates the idea of toes.
5. The Sock Exchange - this name could work for a company that sells socks in a variety of colors and patterns.
6. SoleMate Society - this name incorporates the idea of finding a matching pair of socks, but also implies a sense of community among sock enthusiasts.
7. The Happy Feet Company - this name conveys the idea

'\n\nI\'m looking for something catchy, creative and eye-catching.  I want it to stand out and make people want to buy them!\n\nWhat are some suggestions?\n\nComment: Here are some suggestions for a company name that makes colorful socks:\n\n1. SoleMates - a play on the word "soulmates" that incorporates the idea of matching socks.\n2. Footloose & Fancy Free - this name conveys the idea of being carefree and having fun with your footwear.\n3. Hue & Cry - a play on words that references both color and emotion.\n4. Toe-Tally Awesome - this name is a play on the phrase "totally awesome" and incorporates the idea of toes.\n5. The Sock Exchange - this name could work for a company that sells socks in a variety of colors and patterns.\n6. SoleMate Society - this name incorporates the idea of finding a matching pair of socks, but also implies a sense of community among sock enthusiasts.\n7. The Happy Feet Company - this name conveys the idea'